I am going to be using the training set from the tragic event of the Titanic. I will be using certain attributes from each passenger to predict the survival rate for each passenger. 

At the bottom of the code there will be a prompt to enter attributes about a individual. The end result will be the survival rate of that person on board the Titanic the night it sank. 

(Keep in mind it is still a work in progress, and constantly improving.)

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.ensemble import RandomForestClassifier
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


Placing all pieces of data into DataFrames

In [3]:
train = pd.read_csv('/kaggle/input/titanic/train.csv')
test = pd.read_csv('/kaggle/input/titanic/test.csv')
gender = pd.read_csv('/kaggle/input/titanic/gender_submission.csv')
train.head()
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


The average survival rate for Men on the Titanic

In [4]:
men = train.loc[train.Sex == 'male']['Survived']
rate_men = sum(men)/len(men)
print('% of men who survived: %', rate_men * 100)

% of men who survived: % 18.890814558058924


The average survivial rate for Women on the Titanic

In [5]:
women = train.loc[train.Sex == 'female']['Survived']
rate_women = sum(women)/len(women)
print('% of women who survived: %', rate_women * 100)

% of women who survived: % 74.20382165605095


In this section I will be using the data from the training set to predict if a passenger would survive. The data will include **Sex**, any **Sibsp** (Siblings/Spouses), any **Parch** (Parents/Child), **Passenger Class**, and from where they **Embarked** from.

Then transform the "Sex" column from "Male" and "Female" to Numbers, 
Male = 0
Female = 1

In [6]:
sex = {'male': 0 , 'female': 1}
train.Sex = [sex[item] for item in train.Sex]
train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",0,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",1,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",0,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",0,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",1,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",1,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",0,26.0,0,0,111369,30.0000,C148,C


Next, we will check if there is any Null values in the "Sex" column.

In [7]:
train[train['Sex'].isnull()]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked


Then check the "SibSp" (Sibling and Spouses) column for any null values.

In [8]:
train[train['SibSp'].isnull()]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked


Then check the "Embarked" column has any null values.

In [9]:
train[train['Embarked'].isnull()]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
61,62,1,1,"Icard, Miss. Amelie",1,38.0,0,0,113572,80.0,B28,NaN
829,830,1,1,"Stone, Mrs. George Nelson (Martha Evelyn)",1,62.0,0,0,113572,80.0,B28,NaN


There are two null values in the "Embarked" column. Since it is only two rows within the column that we will remove. It will not effect the overall outcome of the prediction. If there were a significant amount of rows with null values then dropping those rows would not be recommended, because it could lose precious data that could create better accuracies for the predictions. 

Dropping Null values in the "Embarked" column. Since there are only two rows that contain Null values. Should not affect the overall prediction.

In [10]:
train = train.dropna(axis = 0, subset=['Embarked'])

Recheck to see the null values have been dropped from the column.

In [11]:
train[train['Embarked'].isnull()]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked


Transforming the Embarked column from str to integers corresponding to C = 0, Q = 1, S = 2

Then confirming that that the change has happened.

In [12]:
embarked = {'C': 0 , 'Q': 1, 'S': 2}
train.Embarked = [embarked[item] for item in train.Embarked]
train

/opt/conda/lib/python3.7/site-packages/pandas/core/generic.py:5170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",0,22.0,1,0,A/5 21171,7.2500,NaN,2
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38.0,1,0,PC 17599,71.2833,C85,0
2,3,1,3,"Heikkinen, Miss. Laina",1,26.0,0,0,STON/O2. 3101282,7.9250,NaN,2
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35.0,1,0,113803,53.1000,C123,2
4,5,0,3,"Allen, Mr. William Henry",0,35.0,0,0,373450,8.0500,NaN,2
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",0,27.0,0,0,211536,13.0000,NaN,2
887,888,1,1,"Graham, Miss. Margaret Edith",1,19.0,0,0,112053,30.0000,B42,2
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",1,NaN,1,2,W./C. 6607,23.4500,NaN,2
889,890,1,1,"Behr, Mr. Karl Howell",0,26.0,0,0,111369,30.0000,C148,0


Placing data within columns in variable and fitting them in linear_model (regr)

In [13]:
x_train = train[['Pclass', 'SibSp', 'Parch', 'Sex', 'Embarked']]
x_test = test[['Pclass', 'SibSp', 'Parch', 'Sex']]
y_train = train['Survived']
regr = linear_model.LinearRegression()
regr.fit(x_train, y_train)

LinearRegression()

With all data in place. In the "survival_rate" regr.predict() choose number corresponding below to have the prediction.
1. First Number Passenger Class(1=First Class, 2=Middle Class, 3=Lower Class)
2. Second Number Sibling/Spouses(number equals to amount of persons)
3. Third Number Parent/Child(Number equals to amount of persons)
4. Fourth Number Sex(0= Male, 1=Female)
5. Fifth Number Embarked(0= C/Cherbourg, 1= Q/Queenstown, 2=S/Southampton)

In [14]:
survival_rateM = regr.predict([[1, 0, 0, 0, 0]])
print("A male on first class with no siblings or spouse, no parents or children, embarked from Cherbourg has a Survival Rate of %", survival_rateM * 100)

A male on first class with no siblings or spouse, no parents or children, embarked from Cherbourg has a Survival Rate of % [47.34215132]


In [15]:
survival_rateF = regr.predict([[1, 0, 0, 1, 0]])
print("A female on first class with no siblings or spouse, no parents or children, embarked from Cherbourg has a Survival Rate of %",survival_rateF * 100)

A female on first class with no siblings or spouse, no parents or children, embarked from Cherbourg has a Survival Rate of % [99.4506284]


In [27]:
def surv_titanic():
    SurvRate = []
    print("Let's calculate the survival rate of a individual on the tragic event, the Titanic.\nChoose the Passenger Class: First, Second, Third.")
    PClass = input("")
    print(f"This Individual Boarded {PClass} Class")
    if PClass == "1" or "1st" or "First" or "first":
        PClass = 1
        SurvRate.append(PClass)
    elif PClass == "2" or "2nd" or "Second" or "second":
        PClass = 2
        SurvRate.append(PClass)
    elif PClass == "3" or "3rd" or "Third" or "third":
        PClass = 3
        SurvRate.append(PClass)
    else:
        print("Error")
    
    print("Next, How Many Siblings or Spouse: 0 - 3")
    SibSp = input("")
    print(f"This Individual had {SibSp} Siblings or Spouse")

    if SibSp == "1" or "One" or "one" or "Uno" or "uno":
        SibSp = 1
        SurvRate.append(SibSp)
    elif SibSp == "2" or "Two" or "two" or "Dos" or "dos":
        SibSp = 2
        SurvRate.append(SibSp)
    elif SibSp == "3" or "Three" or "three" or "Tres" or "tres":
        SibSp = 3
        SurvRate.append(SibSp)
    elif SibSp == "0" or "Zero" or "zero" or "Cero" or "cero":
        SibSp = 0
        SurvRate.append(SibSp)
    else:
        print("Error")

    print("Next, How Many Children or Parents: 0 - 3")
    ParCh = input("")
    print(f"This Individual had {ParCh} Children or Parents")

    if ParCh == "0":
        ParCh = 0
        SurvRate.append(ParCh)
    elif ParCh == "1":
        ParCh = 1
        SurvRate.append(ParCh)
    elif ParCh == "2":
        ParCh = 2
        SurvRate.append(ParCh)
    elif ParCh == "3":
        ParCh = 3
        SurvRate.append(ParCh)
    else:
        print("Error")
    
    print("Now, are they Male or Female?")
    Sex = input("")
    print(f"You chose {Sex}")

    if Sex == "Male" or "male" or "man" or "Man":
        Sex = 0
        SurvRate.append(Sex)
    elif Sex == "Female" or "female" or "Woman" or "woman":
        Sex = 1
        SurvRate.append(Sex)
    else:
        print("Error")

    print("Lastly, where did this person embark from: 0 or Cherbourg, 1 or Queenstown, 2 or South Hampton")
    Embarked = input("")
    print(f"You chose the individual embarked from {Embarked}")

    if Embarked == "Cherbourg" or "0":
        Embarked = 0
        SurvRate.append(Embarked)
    elif Embarked == "Queenstown" or "1":
        Embarked = 1
        SurvRate.append(Embarked)
    elif Embarked == "South Hampton"or "2":
        Embarked = 2
        SurvRate.append(Embarked)
    else: 
        print("Error")
    
    SRx = regr.predict([SurvRate])
    SRx = str(SRx * 100)[1:-1]
    print("The individual had a %",SRx,"chance of surviving the Titanic.")

In [28]:
surv_titanic()

lets see the survival rate of a individual on the tragic event of the Titanic.
Choose the Passenger Class: First, Second, Third.
Second
This Individual Boarded Second Class
Next, How Many Siblings or Spouse: 0 - 3
1
This Individual had 1 Siblings or Spouse
Next, How Many Children or Parents: 0 - 3
2
This Individual had 2 Children or Parents
Now, are they Male or Female?
Male
You chose Male
Lastly, where did this person embark from: 0 or Cherbourg, 1 or Queenstown, 2 or South Hampton
0
You chose the individual embarked from 0
The individual had a % 43.31638704 chance of surviving the Titanic.


# 